# Описание проекта

В моём распоряжении пробы нефти в трёх регионах. Необходимо, с помощью машинного обучения найти наиболее преспективный с точки зрения прибыли регион для бурения новых скважин.

## Оглавление
[1. Загрузка и подготовка данных](#section_1)<br>
[2. Обучение и проверка модели](#section_2)<br>
[3. Подготовка к расчёту прибыли](#section_3)<br>
[4. Расчёт прибыли по выбранным скважинам и предсказаниям модели](#section_4)<br>
[5. Расчёт рисков и прибыли для каждого региона](#section_5)<br>

<a id='section_1'><h1>1. Загрузка и подготовка данных</h1></a>

In [1]:
#Загрузим необходимые библиотеки
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy import stats as st
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from IPython.display import display
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')

In [2]:
#Загрузим необходимые DataSet'ы
data_0 = pd.read_csv('/datasets/geo_data_0.csv')
data_1 = pd.read_csv('/datasets/geo_data_1.csv')
data_2 = pd.read_csv('/datasets/geo_data_2.csv')

In [3]:
#Посмотрим информацию по всем датасетам
print(data_0.info())
print()
print(data_1.info())
print()
print(data_2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memor

In [4]:
data_0.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [5]:
data_1.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [6]:
data_2.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


В Датасетах нет пропущенных значений. Типы данных изменять не нужно. Также нет необходимости в масштабировании численных признаков.

In [7]:
#Проверим наличие дубликатов в датасетах
print(data_0.duplicated().sum())
print(data_1.duplicated().sum())
print(data_2.duplicated().sum())

0
0
0


Полных дубликатов не обнаружено

In [8]:
#Построимматрицы корреляций для всех датасетов
corr_0 = data_0.corr()
corr_0.style.background_gradient(cmap='cool').set_precision(3)

,f0,f1,f2,product
f0,1,-0.441,-0.00315,0.144
f1,-0.441,1,0.00172,-0.192
f2,-0.00315,0.00172,1,0.484
product,0.144,-0.192,0.484,1


In [9]:
corr_1 = data_1.corr()
corr_1.style.background_gradient(cmap='cool').set_precision(3)

,f0,f1,f2,product
f0,1,0.182,-0.00178,-0.0305
f1,0.182,1,-0.0026,-0.0102
f2,-0.00178,-0.0026,1,0.999
product,-0.0305,-0.0102,0.999,1


In [10]:
corr_2 = data_2.corr()
corr_2.style.background_gradient(cmap='cool').set_precision(3)

,f0,f1,f2,product
f0,1,0.000528,-0.000448,-0.00199
f1,0.000528,1,0.000779,-0.00101
f2,-0.000448,0.000779,1,0.446
product,-0.00199,-0.00101,0.446,1


Мы видим, что во втором датасете есть сильная корреляция между параметром f2 и product. Как я понял, это синтетические данные и удалять этот столбец нет необходимости.

<a id='section_2'><h1>2. Обучение и проверка модели</h1></a>

###### 2.1. Разбитие данных на обучающую и валидационную выборки в соотношении 75:25

In [11]:
data_0_train, data_0_valid = train_test_split(data_0, test_size=0.25, random_state=12345)
data_1_train, data_1_valid = train_test_split(data_1, test_size=0.25, random_state=12345)
data_2_train, data_2_valid = train_test_split(data_2, test_size=0.25, random_state=12345)

##### 2.2. Обучение модели и создание предсказаний на валидационной выборке.

In [12]:
#создадим переменные для фичей и таргета
features_0_train = data_0_train.drop(['product', 'id'], axis=1)
target_0_train = data_0_train['product']
features_0_valid = data_0_valid.drop(['product', 'id'], axis=1)
target_0_valid = data_0_valid['product']

In [13]:
features_1_train = data_1_train.drop(['product', 'id'], axis=1)
target_1_train = data_1_train['product']
features_1_valid = data_1_valid.drop(['product', 'id'], axis=1)
target_1_valid = data_1_valid['product']

In [14]:
features_2_train = data_2_train.drop(['product', 'id'], axis=1)
target_2_train = data_2_train['product']
features_2_valid = data_2_valid.drop(['product', 'id'], axis=1)
target_2_valid = data_2_valid['product']

In [15]:
#Обучим первую модель
model_0 = LinearRegression(normalize=True)
model_0.fit(features_0_train, target_0_train)
model_0_prediction = model_0.predict(features_0_valid)
RMSE_0 = (mean_squared_error(target_0_valid, model_0_prediction))**0.5

In [16]:
#Обучим вторую модель
model_1 = LinearRegression(normalize=True)
model_1.fit(features_1_train, target_1_train)
model_1_prediction = model_1.predict(features_1_valid)
RMSE_1 = (mean_squared_error(target_1_valid, model_1_prediction))**0.5

In [17]:
#Обучим третью модель
model_2 = LinearRegression(normalize=True)
model_2.fit(features_2_train, target_2_train)
model_2_prediction = model_2.predict(features_2_valid)
RMSE_2 = (mean_squared_error(target_2_valid, model_2_prediction))**0.5

##### 2.3. Вывод на экран среднего запаса предсказанного сырья и RMSE моделей

In [18]:
print('Предсказания модели первого региона:', model_0_prediction.mean())
print('RMSE модели первого региона:', RMSE_0)
print()
print('Предсказания модели второго региона:', model_1_prediction.mean())
print('RMSE модели второго региона:', RMSE_1)
print()
print('Предсказания модели третьего региона:', model_2_prediction.mean())
print('RMSE модели третьего региона:', RMSE_2)

Предсказания модели первого региона: 92.59256778438035
RMSE модели первого региона: 37.5794217150813

Предсказания модели второго региона: 68.72854689544602
RMSE модели второго региона: 0.8930992867756158

Предсказания модели третьего региона: 94.96504596800489
RMSE модели третьего региона: 40.02970873393434


##### 2.5. Анализ результатов.

Мы видим, что лучше всего предсказывает подель второго региона (в коде - 1). Эта модель имеет наименьшую RMSE и соответственно точнее. Остальные модели имеют довольно большую RMSE и, соответсвенно, наибольший разброс в предсказаниях.

<a id='section_3'><h1>3. Подготовка к расчёту прибыли</h1></a>

##### 3.1. Сохранение всех ключевых значений для расчётов в отдельных переменных.

In [19]:
#Создадим переменные для расчёта прибыли
budget = 10000000000 #бюджет, выделенный для разработки
unit_income = 450000 #прибыль с единицы товара

##### 3.2. Рассчёт достаточного объёма сырья для безубыточной разработки новой скважины. Сравнение полученного объёма сырья со средним запасом в каждом регионе.

In [20]:
sufficient_volume_well = (budget / unit_income) / 200
sufficient_volume = (budget / unit_income)
print('Минимальный объём скважины, необходимый для разработки:',round(sufficient_volume_well, 2))
print('Необходимый минималный объём 200 скважин:', round(sufficient_volume, 2))

Минимальный объём скважины, необходимый для разработки: 111.11
Необходимый минималный объём 200 скважин: 22222.22


##### 3.3. Вывод по этапу подготовки расчёта прибыли.

Мы получили минимальный объём одной скважины для разработки в 111 тысяч баррелей. Средний объём скважин во всеъ трёх регионах меньше необходимого. Это значит, что нам необходимо выбирать скважины с наибольгшим количеством сырья.

<a id='section_4'><h1>4. Расчёт прибыли по выбранным скважинам и предсказаниям модели</h1></a>

###### 4.1. Выбор скважин с макимальными значениями предсказаний

In [21]:
#Сбросим индекы целевого признака, для соответствия индексам предсказаний
target_0_valid = target_0_valid.reset_index(drop=True)
target_1_valid = target_1_valid.reset_index(drop=True)
target_2_valid = target_2_valid.reset_index(drop=True)
# Изменим тип предсказаний с numpy на Series
prediction_0 = pd.Series(model_0_prediction)
prediction_1 = pd.Series(model_1_prediction)
prediction_2 = pd.Series(model_2_prediction)

In [22]:
#Отсортируем предсказанные значения по убыванию
pred_sorted_0 = prediction_0.sort_values(ascending=False)
pred_sorted_1 = prediction_1.sort_values(ascending=False)
pred_sorted_2 = prediction_2.sort_values(ascending=False)

##### 4.2. Просуммируем целевое значение объёма сырья, соответствующее этим предсказаниям.

In [23]:
#Выберем 200 скважин с наибольшим объёмом
target_volume_0 = target_0_valid[pred_sorted_0.index][:200].sum()
target_volume_1 = target_1_valid[pred_sorted_1.index][:200].sum()
target_volume_2 = target_2_valid[pred_sorted_2.index][:200].sum()

##### 4.3. Рассчёт прибыли для полученного объёма сырья.

In [24]:
#Умножим объём полученного сырья на единицу товара
target_income_0 = target_volume_0 * unit_income
target_income_1 = target_volume_1 * unit_income
target_income_2 = target_volume_2 * unit_income

In [25]:
print('Целевой объём первой области', round(target_volume_0, 2))
print('Целевая прибыль первой области', round(target_income_0, 2))
print('---------------------------------------------------')
print('Целевой объём второй области', round(target_volume_1, 2))
print('Целевая прибыль второй области', round(target_income_1, 2))
print('---------------------------------------------------')
print('Целевой объём третьей области', round(target_volume_2, 2))
print('Целевая прибыль третьей области', round(target_income_2, 2))

Целевой объём первой области 29601.84
Целевая прибыль первой области 13320826043.14
---------------------------------------------------
Целевой объём второй области 27589.08
Целевая прибыль второй области 12415086696.68
---------------------------------------------------
Целевой объём третьей области 28245.22
Целевая прибыль третьей области 12710349963.6


<a id='section_5'><h1>5. Расчёт рисков и прибыли для каждого региона</h1></a>

##### 5.1. Применение техники Bootstrap с 1000 выборок. Нахождение средней прибыли, 95%-го доверительного интервала и риска убытков.

In [32]:
def revenue(target, predictions, count):
    prediction = pd.Series(predictions)
    pred_sorted = prediction.sort_values(ascending=False)
    selected = target[pred_sorted.index][:count]
    return (unit_income * selected.sum()) - budget

state = np.random.RandomState(12345)
values_0 = []
for i in range(1000):
    
    target_subsample = target_0_valid.sample(n=500, replace=True, random_state=state)
    pred_subsample = prediction_0[target_subsample.index]
    
    values_0.append(revenue(target_subsample, pred_subsample, 200))

values_0 = pd.Series(values_0)

mean = values_0.mean()
print("Средняя выручка первого реиона:", (mean / 1000000), 'млн.')


lower = values_0.quantile(0.025) / 1000000
upper = values_0.quantile(0.975) / 1000000


print("95%-ый доверительный интервал первого региона:", lower, 'млн.', upper, 'млн.')

values_0 = values_0
values_0 = values_0[values_0 < 0]
result = (values_0.count() / 1000) 
print('Риск убытков первого региона', format(result, '.2%'))

Средняя выручка первого реиона: 425.9385269105923 млн.
95%-ый доверительный интервал первого региона: -102.09009483793653 млн. 947.976353358369 млн.
Риск убытков первого региона 6.00%


In [34]:
state = np.random.RandomState(12345)
values_1 = []
for i in range(1000):
    
    target_subsample = target_1_valid.sample(n=500, replace=True, random_state=state)
    pred_subsample = prediction_1[target_subsample.index]
    
    values_1.append(revenue(target_subsample, pred_subsample, 200))

values_1 = pd.Series(values_1)

mean = values_1.mean()
print("Средняя выручка второго реиона:", (mean / 1000000), 'млн.')


lower = values_1.quantile(0.025) / 1000000
upper = values_1.quantile(0.975) / 1000000


print("95%-ый доверительный интервал второго региона:", lower, 'млн.', upper, 'млн.')

values_1 = values_1
values_1 = values_1[values_1 < 0]
result = (values_1.count() / 1000) 
print('Риск убытков второго региона', format(result, '.2%'))

Средняя выручка второго реиона: 515.22277344329 млн.
95%-ый доверительный интервал второго региона: 68.87322537050177 млн. 931.5475912570496 млн.
Риск убытков второго региона 1.00%


In [35]:
state = np.random.RandomState(12345)
values_2 = []
for i in range(1000):
    
    target_subsample = target_2_valid.sample(n=500, replace=True, random_state=state)
    pred_subsample = prediction_2[target_subsample.index]
    
    values_2.append(revenue(target_subsample, pred_subsample, 200))

values_2 = pd.Series(values_2)

mean = values_2.mean()
print("Средняя выручка третьего реиона:", (mean / 1000000), 'млн.')


lower = values_2.quantile(0.025) / 1000000
upper = values_2.quantile(0.975) / 1000000


print("95%-ый доверительный интервал третьего региона:", lower, 'млн.', upper, 'млн.')

values_2 = values_2
values_2 = values_2[values_2 < 0]
result = (values_2.count() / 1000) 
print('Риск убытков третьего региона', format(result, '.2%'))

Средняя выручка третьего реиона: 435.00836278275557 млн.
95%-ый доверительный интервал второго региона: -128.880547329789 млн. 969.706954180268 млн.
Риск убытков второго региона 6.40%


##### 5.2. Вывод

Наиболее перспективной является регион номер два (в коде - 1). Этот регион в среднем принесет больше дохода, а также в данном регионе верятность убытков равна 1%. Остальные регионы имеют в среднем меньший доход, а также вероятность убытков более 6%. 